In [1]:
import pandas as pd
import os
import json
root_dir = '../../data'
# for dataset_name in ['linkedln', 'banking', 'clinc', 'mcid', 'reuters', 'stackoverflow', 'thucnews', 'snips', 'atis', 'ele', 'news']:
for dataset_name in ['atis']:
    all_df = []
    for split in ['train', 'eval', 'test']:
        df = pd.read_csv(f'{root_dir}/{dataset_name}/origin_data/{split}.tsv', sep='\t')
        all_df.append(df)
    all_df = pd.concat(all_df)
    all_df['y'] = all_df.groupby('label').ngroup()
    all_df['X'] = all_df['text']
    all_df = all_df[['y', 'X', 'label']].reset_index(drop=True)
    all_df['label'] = all_df['label'].apply(lambda x: x.replace('_', ' '))
    ans = {}
    for col in all_df.columns:
        ans[col] = all_df[col].tolist()
    label2idx = all_df[['y', 'label']].drop_duplicates().sort_values(by=['y'])
    ans['target_names'] = label2idx['label'].tolist()
    label2idx = label2idx.set_index('label').to_dict()['y']
    for ratio in [0.25, 0.5, 0.75]:
        idx = pd.read_csv(f'../../data/{dataset_name}/{dataset_name}_{ratio}/idx.txt', sep='\t')
        idx['index'] = idx['node'].apply(lambda x: label2idx[x])
        ans[f"target_{ratio}"] = idx['index'].tolist()
        for split in ['train', 'test']:
            if split == 'train':
                df = pd.concat([pd.read_csv(f'{root_dir}/{dataset_name}/origin_data/{_}.tsv', sep='\t') for _ in ['train', 'eval']])
            else:
                df = pd.read_csv(f'{root_dir}/{dataset_name}/origin_data/{split}.tsv', sep='\t')

            text_list = df['text'].tolist()
            ans[f"X_{ratio}_{split}"] = all_df[all_df['X'].isin(text_list)].index.tolist()
    json.dump(ans, open(f'data/{dataset_name}.json', 'w'))

KeyError: 'atis cheapest'